# Data Wrangling Plenar Protokols

##https://medium.com/@bedigunjit/simple-guide-to-text-classification-nlp-using-svm-and-naive-bayes-with-python-421db3a72d34

In [1]:
import pandas as pd
import numpy as np
import sklearn.datasets
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
pd.set_option('display.max_columns', None)

In [2]:
df_test = pd.read_csv('/Users/lukas/OneDrive - University of Warwick/Dokumente/Uni/Thesis/Thesis/data/df_base.csv')
df_train = pd.read_csv('/Users/lukas/OneDrive - University of Warwick/Dokumente/Uni/Thesis/Thesis/data/df_train.csv')

In [3]:
print(len(df_test))
print(len(df_train))

562
18240


In [126]:
Train_X = df_train['speechContent']
Train_Y = df_train['Ruling_Party']
Test_X = df_test['speechContent']
Test_Y = df_test['Ruling_Party']

In [128]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/lukas/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [129]:
# Step - a : Remove blank rows if any.
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
Train_X = [entry.lower() for entry in Train_X]
Train_X= [word_tokenize(entry) for entry in Train_X]


NameError: name 'head' is not defined

In [ ]:
# Step - a : Remove blank rows if any.
Train_X.dropna(inplace=True)
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
Train_X = [entry.lower() for entry in Train_X
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
Corpus['text']= [word_tokenize(entry) for entry in Corpus['text']]
# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(Corpus['text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    Corpus.loc[index,'text_final'] = str(Final_words)

In [89]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
nltk.download('stopwords')
german_stop_words = stopwords.words('german')
count_vect = CountVectorizer(stop_words = german_stop_words)
X_train_counts = count_vect.fit_transform(Train_X)
X_train_counts.shape

[nltk_data] Downloading package stopwords to /Users/lukas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(18240, 177309)

In [117]:
print(X_train_counts.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [90]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(18240, 177309)

In [91]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [92]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto', probability=True)
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  87.72241992882563


In [97]:
class_probabilities = SVM.predict_proba(Test_X_Tfidf)

In [110]:
confidence_0 = class_probabilities.transpose()[0]
confidence_1 = class_probabilities.transpose()[1]

In [113]:
df_test['SMV_predictions'] = predictions_SVM
df_test['Confidence_0'] = confidence_0
df_test['Confidence_1'] = confidence_1

In [115]:
df_test.to_csv('/Users/lukas/OneDrive - University of Warwick/Dokumente/Uni/Thesis/Thesis/data/df_base_ML.csv')